In [1]:
import os
from dotenv import load_dotenv
load_dotenv()


True

In [3]:
os.environ["OPENAI_API_KEY"]=os.getenv("OPEN_API_SECRET_KEY")


In [6]:
import pandas as pd
import spacy
from langchain.docstore.document import Document
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# Load SpaCy's English language model for NER
nlp = spacy.load("en_core_web_sm")
csv_file_path = "updated_bank_risk_metadata.csv"
# Load the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# Data Cleaning: Normalize and clean the DataFrame values
df = df.applymap(lambda x: x.strip().lower() if isinstance(x, str) else x)

print("Pandas DataFrame after cleaning:")
print(df)

# Create documents from the cleaned DataFrame
documents = []
for _, row in df.iterrows():
    # Combine fields into a single descriptive text representation
    combined_text = (f"The logical column '{row['logical_column_name']}' corresponds to the physical column '{row['physical_column_name']}' "
                     f"with a data type of '{row['data_type']}', protected under '{row['protection_group']}' group, "
                     f"and belongs to PG group '{row['PGGroup']}'.")
    
    # Named Entity Recognition (NER)
    doc_nlp = nlp(combined_text)
    entities = [(ent.text, ent.label_) for ent in doc_nlp.ents]
    
    # Create Document with combined text and metadata
    metadata = {
        "logical_column_name": row["logical_column_name"],
        "physical_column_name": row["physical_column_name"],
        "data_type": row["data_type"],
        "protection_group": row["protection_group"],
        "PGGroup": row["PGGroup"],
        "entities": entities  # Store recognized entities in metadata
    }
    
    doc = Document(page_content=combined_text, metadata=metadata)
    documents.append(doc)

# Update documents with combined text content
for doc in documents:
    # Extract metadata fields
    table_name = doc.metadata.get('table_name', "")
    logical_column_name = doc.metadata.get('logical_column_name', "")
    physical_column_name = doc.metadata.get('physical_column_name', "")
    data_type = doc.metadata.get('data_type', "")
    protection_group = doc.metadata.get('protection_group', "")
    pg_group = doc.metadata.get('PGGroup', "")

    # Combine fields to create a descriptive text representation for the document
    combined_text = (f"Table: {table_name}, "
                     f"Logical Column: {logical_column_name}, "
                     f"Physical Column: {physical_column_name}, "
                     f"Data Type: {data_type}, "
                     f"Protection Group: {protection_group}, "
                     f"PG Group: {pg_group}")

    # Update the document's content with the combined text
    doc.page_content = combined_text
    print(f"Updated page content: {doc.page_content}")

# Generate embeddings
embedding_adamodel = OpenAIEmbeddings(model="text-embedding-ada-002")
embeddings = [embedding_adamodel.embed_query(doc.page_content) for doc in documents]
metadatas = [doc.metadata for doc in documents]

# Initialize Chroma vector store
vector_store = Chroma(embedding_function=embedding_adamodel, persist_directory="./chroma_db")

from langchain_community.vectorstores.utils import filter_complex_metadata

def filter_metadata(metadata):
    """Filter out complex metadata values."""
    return {k: v for k, v in metadata.items() if isinstance(v, (str, int, float, bool))}

# Use this in your loop where you add texts to the vector store
for doc, embedding, metadata in zip(documents, embeddings, metadatas):
    # Filter the metadata
    filtered_metadata = filter_metadata(metadata)
    
    # Add to vector store
    vector_store.add_texts(texts=[doc.page_content], metadatas=[filtered_metadata], embeddings=[embedding])


C:\Users\sures\AppData\Local\Temp\ipykernel_7244\2837446721.py:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip().lower() if isinstance(x, str) else x)


Pandas DataFrame after cleaning:
     TABLE_NAME              logical_column_name physical_column_name  \
0        branch            branch contact number       branch_contact   
1        branch                  branch location            brnch_loc   
2       account                  branch location            brnch_loc   
3        branch                   branch manager            brnch_mgr   
4       account                   branch manager            brnch_mgr   
5       account                   branch manager            brnch_mgr   
6   credit_card              credit card balance               cc_bal   
7       account              credit card balance               cc_bal   
8   credit_card               credit card expiry               cc_exp   
9       account               credit card expiry               cc_exp   
10  credit_card               credit card number               cc_num   
11      account               credit card number               cc_num   
12     customer   

In [7]:
from langchain_openai import OpenAIEmbeddings
embedding_adamodel=OpenAIEmbeddings(model="text-embedding-ada-002")
embeddings = [embedding_adamodel.embed_query(doc.page_content) for doc in documents]
metadatas=[doc.metadata for doc in documents]

In [8]:
from langchain_chroma import Chroma
# Use metadata for each document (optional)
# Initialize a Chroma vector store (or any other vector store you prefer)
#vector_store = Chroma(embedding_function=embedding_adamodel)

vector_store = Chroma(embedding_function=embedding_adamodel,persist_directory="./chroma_db")

# Store the embeddings in the vector store with their metadata
for doc, embedding, metadata in zip(documents, embeddings, metadatas):
    vector_store.add_texts(texts=[doc.page_content], metadatas=[metadata], embeddings=[embedding])

#store the value


ValueError: Expected metadata value to be a str, int, float or bool, got [] which is a list

Try filtering complex metadata from the document using langchain_community.vectorstores.utils.filter_complex_metadata.

In [64]:
#  Retrieve vector_store
vector_store = Chroma(persist_directory="./chroma_db", embedding_function=embedding_adamodel)

In [67]:
import nltk
#import spacy
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
nltk.download('punkt')
nltk.download('stopwords')
# Initialize SpaCy model

stop_words = set(stopwords.words('english'))
common_phrases = {
    "i want to know", "can you tell me", "could you provide", "what is",
    "please show", "give me", "let me know", "show me", "tell me",
    "provide", "please provide", "what are the details of", "what is the value of",
    "can you list", "how do i find", "what’s the name of", "what do we call",
    "do you know", "would you be able to tell", "how to check", "any details on",
    "what’s the description of", "related to the field", "field", "column",
    "metadata", "attribute", "entry", "name of the field", "physical field",
    "logical field", "column field", "value field", "for the field",
    "placeholder phrases", "regarding the", "based on the", "specific to",
    "related to", "in the field of", "with the field name", "concerning the field",
    "in the column of", "associated with", "terms specific to data types",
    "field data type", "data type for", "format of", "data type", "data format",
    "varchar format", "string format", "numeric value of", "description of",
    "type of", "protection group for", "group name of", "protection level of",
    "pg group for", "protection group name", "security group for", "group type of",
    "group category", "that is called", "like", "such as", "could you tell",
    "could you describe", "any information on", "description of", "what are the",
    "what is the", "list all", "details about","for the field","want", "know", "field"
}



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sures\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sures\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [68]:
def clean_query(query):
    # Normalize the query by converting to lower case
    query = query.lower()
    
    # Remove any special characters and digits
    query = re.sub(r'[^a-zA-Z\s]', '', query)

    # Tokenize the query
    tokens = word_tokenize(query)

    # Remove common phrases and stopwords from the query
    cleaned_tokens = [token for token in tokens if  token not in stop_words and token not in common_phrases]
    #  and
    # Join the cleaned tokens back into a string
    cleaned_query = ' '.join(cleaned_tokens)
    
    return cleaned_query

# Example usage
user_query = "I want to know the physical field name for the field customer closing balance amount"
cleaned_query = clean_query(user_query)

print("Original Query:", user_query)
print("Cleaned Query:", cleaned_query)

Original Query: I want to know the physical field name for the field customer closing balance amount
Cleaned Query: physical name customer closing balance amount


In [69]:
def vector_search(query, vector_store, top_k=250):
    """Perform similarity search based on vector embeddings using space-separated terms."""
    # Ensure query is a string
    if not isinstance(query, str):
        raise ValueError("Query must be a string.")

    # Split the query into individual terms
    terms = query.split()

    # Initialize an empty list to collect results
    all_results = []
    
    query_embedding = embedding_adamodel.embed_query(query)
    results = vector_store.similarity_search_by_vector(query_embedding, k=top_k)
    all_results.extend(results) 
    print(len(all_results))
    # Perform similarity search for each term
    for term in terms:
        print(type(term))
        term=f"logical_column_name:{term}"
        print(term)
        print(type(term))
        query_embedding = embedding_adamodel.embed_query(term)
      # Search in the vector database for the top_k most similar vectors
        results = vector_store.similarity_search_by_vector(query_embedding, k=top_k)
        print(term, results)
        all_results.extend(results)  # Combine results
    print(len(all_results))
    # Remove duplicates by converting to a set, then back to a list
    present = set()
    unique_results = []
    exact_matches = set()  # To track exact matches for logical_column_name
    
    for doc in all_results:
        # Create a unique identifier for each Document
        identifier = (doc.page_content, frozenset(doc.metadata.items()))  # Using frozenset to make metadata hashable

        # Check if it's an exact match on logical_column_name
        logical_column_name = doc.metadata.get('logical_column_name', "")
        if logical_column_name in exact_matches:
            continue  # Skip this document if we already have an exact match

        # Only add the identifier if it hasn't been seen yet
        if identifier not in present:
            present.add(identifier)
            unique_results.append(doc)
            exact_matches.add(logical_column_name)  # Track this exact match

    return unique_results
    
# Example usage


In [70]:

retrieved_results = vector_search(cleaned_query, vector_store)
examples_context = "\n".join([
    f"Logical Column: {doc.metadata['logical_column_name']}, "
    f"Physical Column: {doc.metadata['physical_column_name']}, "
    f"Data Type: {doc.metadata['data_type']}, "
    f"Protection Group: {doc.metadata['protection_group']}, "
    f"PG Group: {doc.metadata['PGGroup']}"
    for doc in retrieved_results  # Iterate over the unique results
])
examples_context

Number of requested results 250 is greater than number of elements in index 33, updating n_results = 33


33
<class 'str'>
logical_column_name:physical
<class 'str'>


Number of requested results 250 is greater than number of elements in index 33, updating n_results = 33


logical_column_name:physical [Document(metadata={'PGGroup': 'PUB', 'data_type': 'Varchar(200)', 'logical_column_name': 'branch location', 'physical_column_name': 'brnch_loc', 'protection_group': 'Public'}, page_content='Table: , Logical Column: branch location, Physical Column: brnch_loc, Data Type: Varchar(200), Protection Group: Public, PG Group: PUB'), Document(metadata={'PGGroup': 'PUB', 'data_type': 'Varchar(200)', 'logical_column_name': 'branch location', 'physical_column_name': 'brnch_loc', 'protection_group': 'Public'}, page_content='Table: , Logical Column: branch location, Physical Column: brnch_loc, Data Type: Varchar(200), Protection Group: Public, PG Group: PUB'), Document(metadata={'PGGroup': 'INTL', 'data_type': 'varchar(255)', 'logical_column_name': 'branch contact number', 'physical_column_name': 'branch_contact', 'protection_group': 'Internal'}, page_content='Table: , Logical Column: branch contact number, Physical Column: branch_contact, Data Type: varchar(255), Prot

Number of requested results 250 is greater than number of elements in index 33, updating n_results = 33
Number of requested results 250 is greater than number of elements in index 33, updating n_results = 33


...

Number of requested results 250 is greater than number of elements in index 33, updating n_results = 33


logical_column_name:balance [Document(metadata={'PGGroup': 'CNFD', 'data_type': 'decimal(20,3)', 'logical_column_name': 'credit card balance', 'physical_column_name': 'cc_bal', 'protection_group': 'Internal'}, page_content='Table: , Logical Column: credit card balance, Physical Column: cc_bal, Data Type: decimal(20,3), Protection Group: Internal, PG Group: CNFD'), Document(metadata={'PGGroup': 'CNFD', 'data_type': 'decimal(20,3)', 'logical_column_name': 'credit card balance', 'physical_column_name': 'cc_bal', 'protection_group': 'Internal'}, page_content='Table: , Logical Column: credit card balance, Physical Column: cc_bal, Data Type: decimal(20,3), Protection Group: Internal, PG Group: CNFD'), Document(metadata={'PGGroup': 'CNFD', 'data_type': 'decimal(20,3)', 'logical_column_name': 'customer opening balance amount', 'physical_column_name': 'cust_open_bal_amt', 'protection_group': 'Confidential'}, page_content='Table: , Logical Column: customer opening balance amount, Physical Column

Number of requested results 250 is greater than number of elements in index 33, updating n_results = 33


logical_column_name:amount [Document(metadata={'PGGroup': 'CNFD', 'data_type': 'decimal(10,2)', 'logical_column_name': 'transaction amount', 'physical_column_name': 'trans_amt', 'protection_group': 'Confidential'}, page_content='Table: , Logical Column: transaction amount, Physical Column: trans_amt, Data Type: decimal(10,2), Protection Group: Confidential, PG Group: CNFD'), Document(metadata={'PGGroup': 'CNFD', 'data_type': 'decimal(10,2)', 'logical_column_name': 'transaction amount', 'physical_column_name': 'trans_amt', 'protection_group': 'Confidential'}, page_content='Table: , Logical Column: transaction amount, Physical Column: trans_amt, Data Type: decimal(10,2), Protection Group: Confidential, PG Group: CNFD'), Document(metadata={'PGGroup': 'CNFD', 'data_type': 'decimal(20,3)', 'logical_column_name': 'loan amount', 'physical_column_name': 'loan_amt', 'protection_group': 'Confidential'}, page_content='Table: , Logical Column: loan amount, Physical Column: loan_amt, Data Type: dec

'Logical Column: customer closing balance amount, Physical Column: cust_cls_bal_amt, Data Type: decimal(20,3), Protection Group: Confidential, PG Group: CNFD\nLogical Column: customer opening balance amount, Physical Column: cust_open_bal_amt, Data Type: decimal(20,3), Protection Group: Confidential, PG Group: CNFD\nLogical Column: credit card balance, Physical Column: cc_bal, Data Type: decimal(20,3), Protection Group: Internal, PG Group: CNFD\nLogical Column: customer credit limit, Physical Column: cust_crdt_lim, Data Type: decimal(20,3), Protection Group: Confidential, PG Group: CNFD\nLogical Column: transaction amount, Physical Column: trans_amt, Data Type: decimal(10,2), Protection Group: Confidential, PG Group: CNFD\nLogical Column: transaction fee, Physical Column: trans_fee, Data Type: decimal(20,3), Protection Group: Confidential, PG Group: CNFD\nLogical Column: loan amount, Physical Column: loan_amt, Data Type: decimal(20,3), Protection Group: Confidential, PG Group: CNFD\nLo

In [71]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
import requests

url = 'https://api.openai.com/v1/chat/completions'
# Set up the headers
api_key = os.environ["OPENAI_API_KEY"]
headers = {
    'Authorization': f'Bearer {api_key}',  # Include your API key in the authorization header
    'Content-Type': 'application/json'      # Specify that we're sending JSON data
}
# Define your prompt template
prompt_template = """
Based on the following examples of logical and physical column names, data types, and protection groups, suggest a new physical column name, data type, protection group, and PG group for the given logical column name. Use the patterns and context provided below.

Existing Metadata Examples:
{examples_context}

New Logical Column: "{logical_column_name}"

Suggest the following fields:
- Physical Column: <suggested_physical_column_name>
- Data Type: <suggested_data_type>
- Protection Group: <suggested_protection_group>
- PG Group: <suggested_pg_group>
"""
def create_prompt(examples_context, logical_column_name):
    return prompt_template.format(examples_context=examples_context, logical_column_name=logical_column_name)

logical_column_name =  cleaned_query

# Create the prompt
formatted_prompt = create_prompt(examples_context, logical_column_name)
# Prepare the payload
payload = {
    'model': 'gpt-3.5-turbo',  # or 'gpt-4'
    'messages': [{'role': 'user', 'content': formatted_prompt}],
    'max_tokens': 150  # Adjust as needed
}

# Make the request
response = requests.post(url, headers=headers, json=payload)

# Check the response
if response.status_code == 200:
    # Extract and print the assistant's response
    assistant_response = response.json()['choices'][0]['message']['content']
    print("Assistant:", assistant_response)
else:
    print(f'Error code: {response.status_code} - {response.json()}')

Assistant: Proposed fields for the new logical column "physical name customer closing balance amount":
- Physical Column: cust_cls_bal_amt_name
- Data Type: decimal(20,3)
- Protection Group: Confidential
- PG Group: CNFD
